In [11]:
import numpy as np
import scipy.io

In [12]:
import scipy.io
import numpy as np

# Cargar archivos
data_x = scipy.io.loadmat('VariableX.mat')
data_xy = scipy.io.loadmat('VariableX_Y.mat')

# Extraer X (5000 muestras)
X = data_x['X'].flatten()

# Extraer Y de forma segura
nombre_clave_y = [k for k in data_xy.keys() if not k.startswith('__')][0]
Y_total = data_xy[nombre_clave_y].flatten()

# IMPORTANTE: Ajustar Y al tamaño de X (las primeras 5000 muestras)
Y = Y_total[:len(X)] 

# Ahora ya no dará error de dimensiones
realizaciones = np.column_stack((X, Y))

print(f"¡Éxito! Datos sincronizados.")
print(f"Tamaño final: {realizaciones.shape}")

¡Éxito! Datos sincronizados.
Tamaño final: (5000, 2)


## 1. Cálculo de entropía

In [13]:
def entropy_estimation(data):
    unique_values, counts = np.unique(data, return_counts=True)
    probabilities = counts / len(data)
    return -np.sum(probabilities * np.log2(probabilities + 1e-12))

## 2. Cálculo de entropía conjunta

In [14]:
def calculo_entropia_conjunta(realizaciones):
    X, Y = realizaciones[:, 0], realizaciones[:, 1]
    alf_x, alf_y = np.unique(X), np.unique(Y)
    
    # Matriz de probabilidades conjuntas
    frec_conjunta = np.zeros((len(alf_x), len(alf_y)))
    for i, x in enumerate(alf_x):
        for j, y in enumerate(alf_y):
            frec_conjunta[i, j] = np.sum((X == x) & (Y == y))
            
    Pxy = frec_conjunta / len(X)
    p_x, p_y = np.sum(Pxy, axis=1), np.sum(Pxy, axis=0)
    eps = 1e-12
    
    HX = -np.sum(p_x * np.log2(p_x + eps))
    HY = -np.sum(p_y * np.log2(p_y + eps))
    HXY = -np.sum(Pxy * np.log2(Pxy + eps))
    
    return alf_x, alf_y, Pxy, HX, HY, HXY

## 3. Cálculo de entropía condicional

In [15]:
# =================================================================
# 3. CÁLCULO DE ENTROPÍA CONDICIONAL
# =================================================================
def calculo_entropia_condicional(realizaciones):
    alf_x, alf_y, Pxy, HX, HY, HXY = calculo_entropia_conjunta(realizaciones)
    p_x, p_y = np.sum(Pxy, axis=1), np.sum(Pxy, axis=0)
    eps = 1e-12
    
    # Matrices de probabilidad condicional P(X|Y) y P(Y|X)
    prob_X_dado_Y = Pxy / (p_y + eps)
    prob_Y_dado_X = (Pxy.T / (p_x + eps)).T
    
    # Entropías condicionales
    entropia_X_dado_Y = HXY - HY
    entropia_Y_dado_X = HXY - HX
    
    return alf_x, alf_y, prob_X_dado_Y, prob_Y_dado_X, entropia_X_dado_Y, entropia_Y_dado_X


## 4. Cálculo información mutua

In [16]:
def calculo_informacion_mutua(realizaciones):
    # I(X;Y) = H(X) + H(Y) - H(X,Y)
    _, _, _, HX, HY, HXY = calculo_entropia_conjunta(realizaciones)
    return HX + HY - HXY

## 5. Capacidad canal 

In [17]:
def capacidad_canal_44(P):
    capacidad_canal = 0
    pX_optima = np.zeros(4)
    paso = 0.05
    
    # Entropía por filas (H(Y|X=xi))
    H_filas = np.zeros(4)
    for i in range(4):
        p_f = P[i, P[i, :] > 0]
        H_filas[i] = -np.sum(p_f * np.log2(p_f))

    for p1 in np.arange(0, 1 + paso, paso):
        for p2 in np.arange(0, 1 - p1 + paso, paso):
            for p3 in np.arange(0, 1 - p1 - p2 + paso, paso):
                p4 = max(0, 1 - p1 - p2 - p3)
                pX = np.array([p1, p2, p3, p4])
                
                pY = np.dot(pX, P)
                pY_v = pY[pY > 1e-12]
                HY = -np.sum(pY_v * np.log2(pY_v))
                HYX = np.dot(pX, H_filas)
                
                if (HY - HYX) > capacidad_canal:
                    capacidad_canal = HY - HYX
                    pX_optima = pX.copy()
    return capacidad_canal, pX_optima

In [18]:
# =================================================================
# EJECUCIÓN Y RESULTADOS FINALES
# =================================================================
# Ejecutamos los cálculos
HX = entropy_estimation(X)
HY = entropy_estimation(Y)
alf_x, alf_y, Pxy, HX_c, HY_c, HXY = calculo_entropia_conjunta(realizaciones)
_, _, P_X_Y, P_Y_X, H_X_Y, H_Y_X = calculo_entropia_condicional(realizaciones)
I_XY = calculo_informacion_mutua(realizaciones)
cap, p_opt = capacidad_canal_44(P_Y_X)

print(f"1. Entropía H(X): {HX:.4f} bits")
print(f"2. Entropía Conjunta H(X,Y): {HXY:.4f} bits")
print(f"3. Entropía Condicional H(X|Y): {H_X_Y:.4f} bits")
print(f"   Entropía Condicional H(Y|X): {H_Y_X:.4f} bits")
print(f"4. Información Mutua I(X;Y): {I_XY:.4f} bits")
print(f"5. Capacidad del Canal: {cap:.4f} bits/símbolo")
print(f"   Distribución óptima p(X): {p_opt}")

1. Entropía H(X): 1.7703 bits
2. Entropía Conjunta H(X,Y): 3.7687 bits
3. Entropía Condicional H(X|Y): 1.7690 bits
   Entropía Condicional H(Y|X): 1.9984 bits
4. Información Mutua I(X;Y): 0.0013 bits
5. Capacidad del Canal: 0.0035 bits/símbolo
   Distribución óptima p(X): [0.  0.  0.5 0.5]
